# VacationPy

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Montes Altos,-5.8333,-47.0667,73.13,92,5,4.81,BR,1612226899
1,Bathsheba,13.2167,-59.5167,78.80,61,20,20.71,BB,1612226899
2,Rikitea,-23.1203,-134.9692,78.80,75,91,14.07,PF,1612226899
3,Moerai,-22.4333,-151.3333,80.33,65,20,7.76,PF,1612226899
4,Torbay,47.6666,-52.7314,28.99,80,90,13.80,CA,1612226900
...,...,...,...,...,...,...,...,...,...
550,Severo-Yeniseyskiy,60.3725,93.0408,-18.36,84,99,2.17,RU,1612227034
551,Havre-St-Pierre,50.2334,-63.5986,6.80,85,1,2.30,CA,1612227034
552,West Bay,19.3667,-81.4167,79.00,73,20,5.75,KY,1612227034
553,Nova Floresta,-6.4553,-36.2033,69.73,85,94,14.09,BR,1612227035


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

# setting locations and weight
cities_df = cities_df.dropna()

locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

# Create a heatmap layer
fig = gmaps.figure(zoom_level=1.6, center=(37,-95))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
    
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [8]:
# create my ideal weather conditions: max temperature < 85 degrees but > 70,
# humidity between 30-50%, and low change of cloudiness.

ideal_weather_df = cities_df.loc[(cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 85) & 
                                 (cities_df["Humidity"] >= 30) & (cities_df["Humidity"] <= 50) & 
                                 (cities_df["Cloudiness"] <= 20)]

# drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna()    

# display dataframe
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,Busselton,-33.6500,115.3333,79.00,50,0,8.93,AU,1612226906
70,Launceston,-41.4500,147.1667,70.00,45,5,6.91,AU,1612226916
150,Hilo,19.7297,-155.0900,80.60,50,20,10.36,US,1612226618
155,Alice Springs,-23.7000,133.8833,84.20,32,0,17.27,AU,1612226937
205,Yeppoon,-23.1333,150.7333,84.20,45,0,5.75,AU,1612226949
289,Comodoro Rivadavia,-45.8667,-67.5000,75.20,43,0,5.75,AR,1612226969
354,Rawson,-43.3002,-65.1023,74.08,30,7,24.43,AR,1612226984
391,Arua,3.0201,30.9111,70.30,33,0,2.77,UG,1612226993
539,Ratnagiri,16.9833,73.3000,75.74,48,0,8.30,IN,1612227031


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# create new hotel dataframe
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# add a new column for Hotel Name and set empty parameters to store data
hotel_df["Hotel Name"] = ""

# set parameters to search for hotels within 5000 meters
parameters = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    # set parameters to loop through each city's coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make API request
    response = requests.get(base_url, params=parameters).json()
    
    # extract search results
    results = response["results"]
    
    # store search results to Hotel Name column
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("no hotel found")
    
hotel_df

,City,Country,Lat,Lng,Hotel Name
29,Busselton,AU,-33.6500,115.3333,Observatory Guest House
70,Launceston,AU,-41.4500,147.1667,Hotel Grand Chancellor Launceston
150,Hilo,US,19.7297,-155.0900,Hilo Hawaiian Hotel
155,Alice Springs,AU,-23.7000,133.8833,Desert Palms Alice Springs
205,Yeppoon,AU,-23.1333,150.7333,Villa Mar Colina
289,Comodoro Rivadavia,AR,-45.8667,-67.5000,Austral Hotel
354,Rawson,AR,-43.3002,-65.1023,Hosteria Sampedro
391,Arua,UG,3.0201,30.9111,Golden Courts Hotel
539,Ratnagiri,IN,16.9833,73.3000,Hotel Landmark


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
<dt>City:</dt><dd>{City}</dd>
<dt>Country:</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
pin_locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Plot hotels on top of the humidity heatmap with pins containing Hotel Name, City, and Country.

# Create a marker_layer using the hotel list to fill the info box
fig = gmaps.figure(zoom_level=1.6, center=(37,-95))
markers = gmaps.marker_layer(pin_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure with hotel names on top of heat map
fig

Figure(layout=FigureLayout(height='420px'))